In [3]:
import json
import sys
import os

with open("training_data/dev.json", "r") as f:
    data = json.load(f)
print(len(data))

200


In [9]:
keys = data.keys()
type(keys)
# get the first key
first_key = next(iter(keys))
print(first_key)

1adc970c-d433-44d0-aa09-d3834986f7a2


In [11]:
data_1 = data[first_key]
print(data_1)

{'Type': 'Single', 'Section_id': 'Results', 'Primary_id': 'NCT00066573', 'Statement': 'there is a 13.2% difference between the results from the two the primary trial cohorts', 'Label': 'Contradiction'}


In [12]:
from dotenv import load_dotenv
load_dotenv()

True

In [17]:
def load_CT(name:str)-> dict:
    with open(os.path.join("training_data","CT json", name+".json"), "r") as f:
        data = json.load(f)
    return data

In [23]:
import instructor
import openai
from atomic_agents.lib.components.agent_memory import AgentMemory
from atomic_agents.agents.base_agent import BaseAgent, BaseAgentConfig, BaseAgentInputSchema
from atomic_agents.lib.components.system_prompt_generator import SystemPromptGenerator

memory = AgentMemory()
client = instructor.from_openai(openai.OpenAI())
system_prompt_generator = SystemPromptGenerator(
    background=["You are a medical reasoning assistant.",
    """You will be given:
- Clinical trial
- Relevant section's name of the trial (Eligibility/Intervention/Results/Adverse Events)
- A statement about the trial""",
"""
Decide if the statement is:
- Entailment: directly supported by the data
- Contradiction: refuted by the data
"""],
    steps=["Read the statement.",
           "Read the relevant section of the trial.",
           "Determine if the statement is supported or refuted by the data."],
    output_instructions=["""Only output **one of the following words**, exactly as shown:
- Entailment
- Contradiction""",
"For example, if the statement is supported by the data, just output 'Entailment'."]
)
agent = BaseAgent(
    config=BaseAgentConfig(
        client=client,
        model="gpt-4o-mini",  # Use your provider's model
        system_prompt_generator=system_prompt_generator,
        memory=memory
    )
)

In [31]:
from atomic_agents.agents.base_agent import BaseAgent, BaseAgentConfig, BaseAgentInputSchema, BaseAgentOutputSchema

input_template = """
Statement:
{$statement$}

Clinical Trial:
{$trial$}

Please determine if the statement is supported or refuted by the trial data.
"""

statement_1 = data_1['Statement']
trial_1 = load_CT(data_1['Primary_id'])
input_text = input_template.replace("{$statement$}", statement_1).replace("{$trial$}", json.dumps(trial_1, indent=2))
print(input_text)
input_schema = BaseAgentInputSchema(
    chat_message=input_text)
response = agent.run(input_schema)
print(response.chat_message)



Statement:
there is a 13.2% difference between the results from the two the primary trial cohorts

Clinical Trial:
{
  "Clinical Trial ID": "NCT00066573",
  "Intervention": [
    "INTERVENTION 1: ",
    "  Exemestane",
    "  Patients receive oral exemestane (25 mg) once daily for 5 years.",
    "  exemestane: Given orally",
    "INTERVENTION 2: ",
    "  Anastrozole",
    "  Patients receive oral anastrozole (1 mg) once daily for 5 years.",
    "  anastrozole: Given orally"
  ],
  "Eligibility": [
    "DISEASE CHARACTERISTICS:",
    "  Histologically confirmed invasive breast cancer",
    "  pT1-3; pNX, pN0-2 or pN3*; M0",
    "  Neoadjuvant patients are eligible no earlier than 3 weeks or later than 3 months after excisional surgery, provided both the clinical-diagnostic staging of cancer and postsurgical resection-pathologic staging of cancer meet the requirements for primary tumor, regional lymph nodes, and distant metastasis classification NOTE: *Only when the sole basis for this

In [32]:
print(response.chat_message)

Contradiction
